In [1]:
%matplotlib inline
import pandas as pd
import nivapy3 as nivapy
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import seaborn as sn
import numpy as np
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.style.use("ggplot")

In [2]:
# Connect to Nivabase
eng = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


# Compare 1995 and 2019 survey results

This notebook generates a one-page summary plot for each parameter in the 1000 Lake survey.

## 1. Stations

Read stations and assign to regions.

In [3]:
# Get stations
stn_df = nivapy.da.select_resa_project_stations([4530], eng)

# Assign acid region IDs
shp_path = r"../data/norway_regions.shp"
reg_gdf = gpd.read_file(shp_path)
stn_df = nivapy.spatial.identify_point_in_polygon(stn_df, reg_gdf, poly_col="name")

print(len(stn_df), "stations in project.")

stn_df.head()

1003 stations in project.


,station_id,station_code,station_name,latitude,longitude,altitude,name
0,3167,620-4-6,Ørteren,60.470,7.795,1147.0,Østlandet
1,3168,621-1-27,Flåvatna,60.200,9.183,855.0,Østlandet
2,3169,621-3-5,Soneren,60.061,9.545,104.0,Østlandet
3,3170,622-2-43,Trytetjern,60.213,9.764,275.0,Østlandet
4,3171,622-4-4,Krøderen,60.327,9.645,133.0,Østlandet


## 2. Get parameters measured in 2019

In [4]:
# Time period of interest
st_dt = "2019-08-01"
end_dt = "2019-12-31"

par_df = nivapy.da.select_resa_station_parameters(
    stn_df["station_id"].unique(), st_dt, end_dt, eng
)

par_df.head()

36 parameters available for the selected stations and dates.


,parameter_id,parameter_name,unit
0,3,ALK,mmol/l
1,50,Al,µg/l
2,55,Al/Il,µg/l
3,48,Al/R,µg/l
4,223,As,µg/l


## 3. Get water chemistry

In [5]:
# Data from 1995
wc_95, dups_95 = nivapy.da.select_resa_water_chemistry(
    stn_df["station_id"].unique(),
    par_df,
    "1995-08-01",
    "1995-12-31",
    eng,
    drop_dups=True,
)

# Data from 2019
wc_19, dups_19 = nivapy.da.select_resa_water_chemistry(
    stn_df["station_id"].unique(), par_df, st_dt, end_dt, eng, drop_dups=True
)

# Get the "1000 Lakes" samples
# Only surface samples
wc_95 = wc_95.query("(depth1 == 0) and (depth2 == 0)")
wc_19 = wc_19.query("(depth1 == 0) and (depth2 == 0)")
del wc_95["depth1"], wc_95["depth2"], wc_19["depth1"], wc_19["depth2"]

# F is only measured for 1000 Lakes
wc_95.dropna(subset=["F_µg/l"], inplace=True)
wc_19.dropna(subset=["F_µg/l"], inplace=True)

assert len(wc_19) == 1001

# Remove Åsmundvatnet as sample contaminated with sea water
wc_19 = wc_19.query("station_code != '1630-1-19'")

# Calculate averages for stations with multiple samples
idx_cols = ["station_id", "station_code", "station_name"]
wc_95 = wc_95.groupby(idx_cols).mean().reset_index()
wc_19 = wc_19.groupby(idx_cols).mean().reset_index()

print(len(wc_95), "samples in 1995.")
print(len(wc_19), "samples in 2019.")

WARNING
The database contains unexpected duplicate values for some station-date-parameter combinations.
Only the most recent values will be used, but you should check the repeated values are not errors.
The duplicated entries are returned in a separate dataframe.

985 samples in 1995.
1000 samples in 2019.


## 4. Calculate derived parameters

### 4.1. LAL

In [6]:
# Calculate LAL
wc_95["LAL_µg/l"] = wc_95["Al/R_µg/l"] - wc_95["Al/Il_µg/l"]
wc_19["LAL_µg/l"] = wc_19["Al/R_µg/l"] - wc_19["Al/Il_µg/l"]

### 4.2. ALK-E

See e-mail from Øyvind G received 12.06.2020 at 16:33 for the equation:

$$ALKE = (ALK * 1000 - 31.6) + 0.646(ALK * 1000 - 31.6)^{0.5}$$

In [7]:
# Calculate ALKE
wc_95["ALKE_µeq/l"] = (wc_95["ALK_mmol/l"] * 1000 - 31.6) + 0.646 * (
    (wc_95["ALK_mmol/l"] * 1000 - 31.6) ** 0.5
)
wc_95["ALKE_µeq/l"].fillna(0, inplace=True)

wc_19["ALKE_µeq/l"] = (wc_19["ALK_mmol/l"] * 1000 - 31.6) + 0.646 * (
    (wc_19["ALK_mmol/l"] * 1000 - 31.6) ** 0.5
)
wc_19["ALKE_µeq/l"].fillna(0, inplace=True)

### 4.3. ANC

In [8]:
def calculate_anc(df):
    """ Calculate ANC based on data in RESA.
    
            ANC = (ECa+EMg+EK+ENa+ENH4) - (ECl+ESO4+ENO3)
            
    Args:
        df: Dataframe. As returned by nivapy.da.select_resa_water_chemistry()
        
    Returns:
        The dataframe is returned with a new column, 'ANC_µeq/l', added.
    """
    # Tabulate chemical properties
    chem_dict = {
        "molar_mass": [96.06, 35.45, 40.08, 24.31, 14.01, 39.10, 22.99, 14.01],
        "valency": [2, 1, 2, 2, 1, 1, 1, 1],
        "resa2_ref_ratio": [0.103, 1.0, 0.037, 0.196, "N/A", 0.018, 0.859, "N/A"],
    }

    chem_df = pd.DataFrame(
        chem_dict,
        index=[
            "SO4_mg/l",
            "Cl_mg/l",
            "Ca_mg/l",
            "Mg_mg/l",
            "NO3-N_µg/l N",
            "K_mg/l",
            "Na_mg/l",
            "NH4-N_µg/l N",
        ],
    )
    chem_df = chem_df[["molar_mass", "valency", "resa2_ref_ratio"]]

    # Convert to microequivalents
    for par in [
        "SO4_mg/l",
        "Cl_mg/l",
        "Ca_mg/l",
        "Mg_mg/l",
        "NO3-N_µg/l N",
        "K_mg/l",
        "Na_mg/l",
        "NH4-N_µg/l N",
    ]:
        val = chem_df.loc[par, "valency"]
        mm = chem_df.loc[par, "molar_mass"]

        if par == "NO3-N_µg/l N":
            df["ENO3_µeq/l"] = df[par] * val / mm
        elif par == "NH4-N_µg/l N":
            df["ENH4_µeq/l"] = df[par] * val / mm
        else:
            name = par.split("_")[0]
            df[f"E{name}_µeq/l"] = df[par] * val * 1000.0 / mm

    #    # Apply sea-salt correction
    #    for par in ['ESO4_µeq/l', 'EMg_µeq/l', 'ECa_µeq/l']:#, 'ENa_µeq/l', 'EK_µeq/l']:
    #        ref = chem_df.loc[par.split('_')[0][1:]+'_mg/l', 'resa2_ref_ratio']
    #        df[par] = df[par] - (ref*df['ECl_µeq/l'])

    # Calculate ANC = (ECa+EMg+EK+ENa+ENH4) - (ECl+ESO4+ENO3)
    df["ANC_µeq/l"] = (
        df["ECa_µeq/l"]
        + df["EMg_µeq/l"]
        + df["EK_µeq/l"]
        + df["ENa_µeq/l"]
        + df["ENH4_µeq/l"]
    ) - (df["ECl_µeq/l"] + df["ESO4_µeq/l"] + df["ENO3_µeq/l"])

    # Remove intermediates
    df.drop(
        [
            "ESO4_µeq/l",
            "ECl_µeq/l",
            "EMg_µeq/l",
            "ECa_µeq/l",
            "ENO3_µeq/l",
            "EK_µeq/l",
            "ENa_µeq/l",
            "ENH4_µeq/l",
        ],
        axis=1,
        inplace=True,
    )

    return df

In [9]:
# Calculate ANC
wc_95 = calculate_anc(wc_95)
wc_19 = calculate_anc(wc_19)

## 4. Plot data

In [10]:
def plot_points(
    df,
    par,
    ax,
    lat="latitude",
    lon="longitude",
    cmap="coolwarm",
    s=10,
    title=None,
    vmin=None,
    vmax=None,
    vmax_pct=0.95,
    cbar_ticks=None,
    cbar_labels=None,
):
    """ Plot Norwegian point data on specified axis.
    """

    if vmax and vmax_pct:
        raise ValueError("Please specify 'vmax' or 'vmax_pct', not both.")

    # Plot Norway
    shp = cartopy.io.shapereader.natural_earth(
        resolution="50m", category="cultural", name="admin_0_countries"
    )
    reader = cartopy.io.shapereader.Reader(shp)
    countries = reader.records()

    # Add Norway outline
    for country in countries:
        if country.attributes["NAME"] == "Norway":
            ax.add_geometries(
                [country.geometry],
                ccrs.PlateCarree(),  # CRS of Natural Earth data
                facecolor="none",
                edgecolor="black",
                linewidth=1,
                zorder=5,
            )

    if vmax_pct:
        vmax = df[par].describe(percentiles=[0.95,]).loc["95%"]

    # Add points using linear colour ramp from 0 to vmax
    pts = ax.scatter(
        df[lon].values,
        df[lat].values,
        c=df[par].values,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        s=s,
        zorder=5,
        edgecolors="none",
        transform=ccrs.PlateCarree(),
    )

    # Add axis for colourbar
    cax = inset_axes(
        ax,
        width="5%",
        height="30%",
        loc="lower left",
        bbox_to_anchor=(0.8, 0.05, 1, 1),
        bbox_transform=ax.transAxes,
        borderpad=0,
    )

    # Add colourbar
    cbar = fig.colorbar(pts, cax=cax, ticks=cbar_ticks)
    if cbar_labels:
        cbar.ax.set_yticklabels(cbar_labels)

    # Turn off border around subplot
    ax.outline_patch.set_edgecolor("white")

    if title:
        ax.set_title(title, fontsize=12, loc="left")

In [11]:
# Set scale choices for scatterplots
log_axis_scale_dict = {
    "ALK": False,
    "ALKE": False,
    "Al/Il": False,
    "Al/R": False,
    "As": False,
    "Ca": False,
    "Cd": False,
    "Cl": False,
    "Co": False,
    "Cr": False,
    "Cu": False,
    "F": False,
    "Fe": False,
    "KOND": False,
    "K": False,
    "Mg": False,
    "Mn": False,
    "NH4-N": False,
    "NO3-N": False,
    "Na": False,
    "Ni": False,
    "Pb": False,
    "SO4": False,
    "SiO2": False,
    "TOC": False,
    "TOTN": False,
    "TOTP": False,
    "Zn": False,
    "pH": False,
    "LAL": False,
    "ANC": False,
}

In [18]:
# Loop over parameters
for col in wc_19.columns:
    if col in wc_95.columns:
        if col not in idx_cols:
            par, unit = col.split("_")
            print("Processing:", par)

            # Get 2019 data
            df19 = wc_19[["station_id", col]].dropna()
            nvals = len(df19)

            # Join co-ords
            df19 = pd.merge(
                df19,
                stn_df[["station_id", "latitude", "longitude", "name"]],
                how="left",
                on="station_id",
            )

            df9519 = pd.merge(
                df19,
                wc_95[["station_id", col]],
                how="left",
                on="station_id",
                suffixes=["_19", "_95"],
            ).dropna()

            # Calc ratio 2019:1995
            df9519["ratio_2019:1995"] = df9519[col + "_19"] / df9519[col + "_95"]
            df9519["log(ratio_2019:1995)"] = np.log10(df9519["ratio_2019:1995"])

            # Melt to lon
            melt_df = df9519[["station_id", "name", col + "_95", col + "_19"]].copy()
            melt_df.columns = ["station_id", "Region", "1995", "2019"]
            melt_df = pd.melt(
                melt_df,
                id_vars=["station_id", "Region"],
                var_name="Year",
                value_name=f"{par} ({unit})",
            )

            # Define co-ord system
            crs = ccrs.AlbersEqualArea(
                central_longitude=15,
                central_latitude=65,
                false_easting=650000,
                false_northing=800000,
                standard_parallels=(55, 75),
            )

            # Setup plot template
            fig = plt.figure(figsize=(12, 15))
            gs = fig.add_gridspec(ncols=2, nrows=5)
            ax1 = fig.add_subplot(gs[0:3, 0], projection=crs)
            ax1.set_extent([0, 1300000, 0, 1600000], crs=crs)

            ax2 = fig.add_subplot(gs[0:3, 1], projection=crs)
            ax2.set_extent([0, 1300000, 0, 1600000], crs=crs)

            #            ax3 = fig.add_subplot(gs[2, 0])
            #            ax4 = fig.add_subplot(gs[2, 1])
            ax3 = fig.add_subplot(gs[3:, :], aspect="equal")

            # Plot values in 2019
            if par == "pH":
                title = f"(a) {par} in 2019 (n = {nvals})"
            else:
                title = f"(a) {par} in 2019 ({unit}; n = {nvals})"

            plot_points(df19, par=col, ax=ax1, title=title)

            # Plot ratio of values 2019:1995
            if par == "pH":
                # Plot pH ratios directly
                plot_points(
                    df9519,
                    par="ratio_2019:1995",
                    ax=ax2,
                    # vmin=-1,
                    # vmax=1,
                    # vmax_pct=None,
                    title="(b) Value 2019 / Value 1995",
                    # cbar_ticks=[-1, -0.7, -0.3, 0, 0.3, 0.7, 1],
                    # cbar_labels=["÷10", "÷5", "÷2", "Equal", "x2", "x5", "x10"],
                )
            else:
                # Plor log ratio
                plot_points(
                    df9519,
                    par="log(ratio_2019:1995)",
                    ax=ax2,
                    vmin=-1,
                    vmax=1,
                    vmax_pct=None,
                    title="(b) log(Value 2019 / Value 1995)",
                    cbar_ticks=[-1, -0.7, -0.3, 0, 0.3, 0.7, 1],
                    cbar_labels=["÷10", "÷5", "÷2", "Equal", "x2", "x5", "x10"],
                )

            #            # Plot CDF
            #            # If IQR of data is zero (e.g. because most values are LOD), Seaborn fails to
            #            # select bandwidth for KDE. See https://github.com/mwaskom/seaborn/issues/1990
            #            # Set bandwidth for F in 1995 manually
            #            if par == "F":
            #                sn.distplot(
            #                    df9519[col + "_95"],
            #                    hist=False,
            #                    kde_kws={"cumulative": True, "bw": 0.1},
            #                    ax=ax3,
            #                    label="1995",
            #                )
            #
            #            else:
            #                sn.distplot(
            #                    df9519[col + "_95"],
            #                    hist=False,
            #                    kde_kws={"cumulative": True},
            #                    ax=ax3,
            #                    label="1995",
            #                )
            #
            #            sn.distplot(
            #                df9519[col + "_19"].values,
            #                hist=False,
            #                kde_kws={"cumulative": True},
            #                ax=ax3,
            #                label="2019",
            #            )
            #
            #            ax3.set_title(
            #                "(c) Cumulative distribution functions", fontsize=12, loc="left"
            #            )
            #            ax3.set_xlabel(f"{par} ({unit})", fontsize=10)
            #            ax3.set_ylabel("Probability", fontsize=10)

            #            # Scatter plot
            #            ax3.plot(
            #                df9519[col + "_95"],
            #                df9519[col + "_19"],
            #                c=df9519["name"],
            #                fillstyle="none",
            #                markeredgewidth=0.5,
            #            )

            # Scatter plot
            df9519.rename({"name": "Region"}, axis=1, inplace=True)
            sn.scatterplot(
                x=df9519[col + "_95"],
                y=df9519[col + "_19"],
                hue=df9519["Region"],
                ax=ax3,
            )

            ax3.plot(df9519[col + "_95"], df9519[col + "_95"], "k--", label="1:1 line")

            if log_axis_scale_dict[par]:
                ax3.set(xscale="log", yscale="log")

            # ax3.legend(loc="best")
            ax3.set_title("(c) Scatter plot", fontsize=12, loc="left")
            ax3.set_xlabel(f"{par} in 1995 ({unit})", fontsize=10)
            ax3.set_ylabel(f"{par} in 2019 ({unit})", fontsize=10)

            #            # Ensure plot is square
            #            ax_min = min(ax3.get_xlim()[0], ax3.get_ylim()[0])
            #            ax_max = max(ax3.get_xlim()[1], ax3.get_ylim()[1])
            #            ax3.set_xlim((ax_min, ax_max))
            #            ax3.set_ylim((ax_min, ax_max))

            #            # Box plots
            #            box = sn.violinplot(
            #                data=melt_df, x="Region", y=f"{par} ({unit})", hue="Year", ax=ax5
            #            )
            #            box.set_xticklabels(box.get_xticklabels(), rotation=30, ha="right")
            #            ax5.set_title("(e) Violin plots", fontsize=12, loc="left")

            plt.suptitle(par, fontsize=20, y=0.9, fontweight="bold")
            plt.subplots_adjust(hspace=0)

            # Save
            png_path = f"../output/summary_plots/1000_lakes_2019_{par.replace('/', '-').lower()}.png"
            plt.savefig(png_path, dpi=200)
            plt.close()

Processing: ALK
Processing: Al/Il
Processing: Al/R
Processing: As


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing: Ca
Processing: Cd
Processing: Cl
Processing: Co
Processing: Cr
Processing: Cu
Processing: F
Processing: Fe
Processing: KOND
Processing: K
Processing: Mg
Processing: Mn
Processing: NH4-N
Processing: NO3-N
Processing: Na
Processing: Ni
Processing: Pb
Processing: SO4
Processing: SiO2
Processing: TOC
Processing: TOTN
Processing: TOTP
Processing: Zn
Processing: pH
Processing: LAL


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing: ALKE
Processing: ANC
